# Design pipeline
- design python based data processing python for analysis of kauai source receptions at OOI hydrophones

**current problems**
- [ ] script fails for partial data over the two hours
- [ ] something is breaking with the transmission cooridinate not matching the Tx_times variable

In [1]:
import xarray as xr
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import ooipy
from datetime import datetime
from tqdm import tqdm
import kaooi
import hvplot.xarray
import obspy
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
import zarr
import os
import xrsignal

%matplotlib inline
%config InlineBackend.figure_format = 'svg'

%load_ext autoreload
%autoreload 2

## look into seismometers
because why not?

In [7]:
starttime = datetime(2023,4,1)
endtime = datetime(2023,4,1,1)



nodes = ['AXAS1','AXAS2','AXBA1','AXCC1','AXCC2','AXEC1','AXEC2','AXEC3','AXID1','HYS11','HYS12','HYS13','HYS14','HYSB1']

hdata = ooipy.get_acoustic_data_LF(starttime, endtime, node='AXBA1', channel='HHZ')

In [9]:
hdata.stats

         network: OO
         station: AXBA1
        location: AXBA1
         channel: HHZ
       starttime: 2023-04-01T00:00:00.000000Z
         endtime: 2023-04-01T01:00:00.000000Z
   sampling_rate: 200.0
           delta: 0.005
            npts: 720001
           calib: 1.0
         _format: MSEED
           mseed: AttribDict({'dataquality': 'M', 'number_of_records': 1835, 'encoding': 'STEIM2', 'byteorder': '>', 'record_length': 512, 'filesize': 939520})

## test dataset (netCDFs local)

In [27]:
ds = xr.open_mfdataset('/datadrive/kauai/transmissions/*.nc')

ds = ds.assign_coords({'time':np.arange(0,3600*2 + 1/1000, 1/1000)})

In [35]:
ds['AXCC1']

<xarray.DataArray 'AXCC1' (time: 7200001, transmission: 7)>
dask.array<concatenate, shape=(7200001, 7), dtype=float64, chunksize=(7200001, 1), chunktype=numpy.ndarray>
Coordinates:
  * transmission  (transmission) datetime64[ns] 2023-03-18 ... 2023-03-26
  * time          (time) float64 0.0 0.001 0.002 ... 7.2e+03 7.2e+03 7.2e+03
Attributes: (12/13)
    sampling_rate:  1000.0
    starttime:      2023-03-18T00:00:00.000000Z
    npts:           7200001
    calib:          1.0
    network:        OO
    station:        AXCC1
    ...             ...
    channel:        HDH
    mseed:          AttribDict({'dataquality': 'M', 'number_of_records': 4116...
    _format:        MSEED
    processing:     ["ObsPy 1.4.0: resample(no_filter=True::sampling_rate=100...
    delta:          0.001
    endtime:        2023-03-18T02:00:00.000000Z

In [49]:
spec = xrsignal.welch(ds['LJ01C'].chunk({'time':5000, 'transmission':1})[:7200000], dim='time', nperseg=256).compute()

In [50]:
spec = spec.assign_coords({'time':np.arange(0,3600*2, 5)})

In [51]:
(10*np.log10(spec[:,1,:])).hvplot.image(x='time', y='time_frequency', rasterize=True, cmap='inferno')

:DynamicMap   []
   :Image   [time,time_frequency]   (psd across time dimension)

In [2]:
Tx_times = kaooi.get_Tx_keytimes(2023)

In [3]:
hdata1 = kaooi.get_ooi_data(Tx_times[1], length='10s', verbose=True)
hdata2 = kaooi.get_ooi_data(Tx_times[2], length='10s', verbose=True)

LJ01D:
Fetching URLs...
Client response:  https://rawdata.oceanobservatories.org/files/CE02SHBP/LJ01D/11-HYDBBA106/2023/03/18/
No data available for specified day and node. Please change the day or use a different node
PC01A:
Fetching URLs...
Sorting valid URLs for Time Window...


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:03<00:00,  1.03it/s]


Merging Data...
PC03A:
Fetching URLs...
Sorting valid URLs for Time Window...


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:03<00:00,  1.09it/s]


Merging Data...
LJ01C:
Fetching URLs...
Sorting valid URLs for Time Window...


...

Merging Data...
LJ01C:
Fetching URLs...
Sorting valid URLs for Time Window...


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:03<00:00,  1.17it/s]


Merging Data...
LJ03A:
Fetching URLs...
Sorting valid URLs for Time Window...


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:03<00:00,  1.18it/s]


Merging Data...
AXBA1:
AXCC1:
AXEC2:
HYS14:
HYSB1:


In [14]:
hdata1.to_zarr('test.zarr', mode='w')
hdata2.to_zarr('test.zarr', append_dim='transmission')

/home/jhrag/anaconda3/envs/ooi_dev/lib/python3.10/site-packages/xarray/core/dataset.py:2461: SerializationWarning: saving variable None with floating point data as an integer dtype without any _FillValue to use for NaNs
  return to_zarr(  # type: ignore[call-overload,misc]


In [15]:
ds_open = xr.open_zarr('test.zarr')

In [16]:
ds_open

<xarray.Dataset>
Dimensions:       (time: 10001, transmission: 2)
Coordinates:
  * transmission  (transmission) datetime64[ns] 2023-03-18T04:00:00 2023-03-1...
Dimensions without coordinates: time
Data variables:
    AXBA1         (time, transmission) float64 dask.array<chunksize=(10001, 1), meta=np.ndarray>
    AXCC1         (time, transmission) float64 dask.array<chunksize=(10001, 1), meta=np.ndarray>
    AXEC2         (time, transmission) float64 dask.array<chunksize=(10001, 1), meta=np.ndarray>
    HYS14         (time, transmission) float64 dask.array<chunksize=(10001, 1), meta=np.ndarray>
    HYSB1         (time, transmission) float64 dask.array<chunksize=(10001, 1), meta=np.ndarray>
    LJ01C         (time, transmission) float64 dask.array<chunksize=(10001, 1), meta=np.ndarray>
    LJ01D         (time, transmission) float64 dask.array<chunksize=(10001, 1), meta=np.ndarray>
    LJ03A         (time, transmission) float64 dask.array<chunksize=(10001, 1), meta=np.ndarray>
    PC01A         (time, transmission) float64 dask.array<chunksize=(10001, 1), meta=np.ndarray>
    PC03A         (time, transmission) float64 dask.array<chunksize=(10001, 1), meta=np.ndarray>

- upload to azure blob storage

In [59]:
connection_string = os.environ['AZURE_CONSTR_ooidata']
blob_service_client = BlobServiceClient.from_connection_string(connection_string)
container_name = "kauai"
container_client = blob_service_client.get_container_client(container_name)

In [ ]:
# Create the Zarr store using ABSStore
output_path = 'kauai_transmissions_ooi.zarr'

zarr_store = zarr.storage.ABSStore(
    client = container_client,
    prefix=output_path,
)

hdata.to_zarr(store=zarr_store, mode="w-")

### fix bug of partial data

In [10]:
Tx_times = kaooi.get_Tx_keytimes(2023)

In [ ]:
starttime = datetime.datetime(2023,3,

## test dataset (azure zarr)

In [15]:
account_key= os.environ['AZURE_KEY_ooidata']
storage_options={'account_name': 'ooidata', 'account_key': account_key}
ds = xr.open_zarr('abfs://kauai/kauai_transmissions_ooi.zarr', storage_options=storage_options)

In [16]:
ds = ds.assign_coords({'time':np.arange(0,3600*2 + 1/1000, 1/1000)})

In [17]:
ds.transmission

<xarray.DataArray 'transmission' (transmission: 13)>
array(['2023-03-18T00:00:00.000000000', '2023-03-18T00:00:00.000000000',
       '2023-03-18T00:00:00.000000000', '2023-03-18T00:00:00.000000000',
       '2023-03-19T00:00:00.000000000', '2023-03-19T00:00:00.000000000',
       '2023-03-26T00:00:00.000000000', '2023-03-26T00:00:00.000000000',
       '2023-03-26T00:00:00.000000000', '2023-03-26T00:00:00.000000000',
       '2023-03-27T00:00:00.000000000', '2023-03-27T00:00:00.000000000',
       '2023-03-30T00:00:00.000000000'], dtype='datetime64[ns]')
Coordinates:
  * transmission  (transmission) datetime64[ns] 2023-03-18 ... 2023-03-30

In [13]:
Tx_times[:13]

[Timestamp('2023-03-18 00:00:00'),
 Timestamp('2023-03-18 04:00:00'),
 Timestamp('2023-03-18 08:00:00'),
 Timestamp('2023-03-18 12:00:00'),
 Timestamp('2023-03-18 16:00:00'),
 Timestamp('2023-03-18 20:00:00'),
 Timestamp('2023-03-26 00:00:00'),
 Timestamp('2023-03-26 04:00:00'),
 Timestamp('2023-03-26 08:00:00'),
 Timestamp('2023-03-26 12:00:00'),
 Timestamp('2023-03-26 16:00:00'),
 Timestamp('2023-03-26 20:00:00'),
 Timestamp('2023-03-30 00:00:00')]

## Look into ooipy merging

In [12]:
starttime = kaooi.get_Tx_keytimes(2023)[0]
endtime = starttime + pd.Timedelta('10S')
node='LJ01A'

In [13]:
st = ooipy.get_acoustic_data(starttime, endtime, node, verbose=True)

Fetching URLs...
Sorting valid URLs for Time Window...


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:00<00:00,  9.64it/s]


In [16]:
starttime

Timestamp('2023-03-18 00:00:00')